# **Laboratorio 11: LLM y Agentes Autónomos 🤖**

MDS7202: Laboratorio de Programación Científica para Ciencia de Datos

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### **Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados**

- Nombre de alumno 1: Carolina Nuñez
- Nombre de alumno 2: Alonso Uribe

### **Link de repositorio de GitHub:** [Repositorio💻](https://github.com/carinunez/Labs_MDS)

## **Temas a tratar**

- Reinforcement Learning
- Large Language Models

## **Reglas:**

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

### **Objetivos principales del laboratorio**

- Resolución de problemas secuenciales usando Reinforcement Learning
- Habilitar un Chatbot para entregar respuestas útiles usando Large Language Models.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

## **1. Reinforcement Learning (2.0 puntos)**

En esta sección van a usar métodos de RL para resolver dos problemas interesantes: `Blackjack` y `LunarLander`.

In [1]:
!pip install -qqq gymnasium stable_baselines3
!pip install -qqq swig
!pip install -qqq gymnasium[box2d]

### **1.1 Blackjack (1.0 puntos)**

<p align="center">
  <img src="https://www.recreoviral.com/wp-content/uploads/2016/08/s3.amazonaws.com-Math.gif"
" width="400">
</p>

La idea de esta subsección es que puedan implementar métodos de RL y así generar una estrategia para jugar el clásico juego Blackjack y de paso puedan ~~hacerse millonarios~~ aprender a resolver problemas mediante RL.

Comencemos primero preparando el ambiente. El siguiente bloque de código transforma las observaciones del ambiente a `np.array`:


In [2]:
import gymnasium as gym
from gymnasium.spaces import MultiDiscrete
import numpy as np

class FlattenObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super(FlattenObservation, self).__init__(env)
        self.observation_space = MultiDiscrete(np.array([32, 11, 2]))

    def observation(self, observation):
        return np.array(observation).flatten()

# Create and wrap the environment
env = gym.make("Blackjack-v1")


In [3]:
env = FlattenObservation(env)

In [4]:
print("Action Space:", env.action_space)
print("Obs Space:", env.observation_space)

Action Space: Discrete(2)
Obs Space: MultiDiscrete([32 11  2])


In [5]:
print(env.reset())


(array([21,  5,  1]), {})


In [6]:
action = env.action_space.sample()
print("Sample Action:", action)
print("Sample Observation:", env.step(1))

Sample Action: 0
Sample Observation: (array([21,  5,  0]), 0.0, False, False, {})


#### **1.1.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [Blackjack](https://gymnasium.farama.org/environments/toy_text/blackjack/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas.

Las observaciones entregadas se componen de una tupla, en seguida se describen por posición.  
Los estados son discretos y se tienen $32 \cdot 11 \cdot 2$ en total correspondiente a todas las combinaciones de estas tres variables.
* 0: suma de las dos cartas iniciales del jugador
* 1: valor de carta boca arriba del repartidor (1-10 con un Ace)
* 2: Si el jugador posee un Ace

La acción es un valor 0 o 1 (int)
* 0: Seguir sacando cartas para acercarse a la suma 21
* 1: Retirarse

Las recompensas (int)
* 1: gana
* 0: empata
* -1: pierde
* 1.5: ganando con Blackjack natural (Ace + J, Q o K). Opcional.

Si el juego ha terminado (bool); ganando, perdiendo o empatando

#### **1.1.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 5000 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política? ¿Cómo podría interpretar las recompensas obtenidas?

In [7]:
import pandas as pd

In [8]:
epochs = 5000
rewards = []
for epoch in range(epochs):
    obs, _ = env.reset()
    done = False
    while not done:
        action = np.random.randint(0,2) # acción aleatoria
        obs, reward, done, _, _ = env.step(action)
    # Solo queremos la recompensa
    rewards.append(reward)

In [9]:
rewards = pd.Series(rewards)
print("Media:", rewards.mean())
print("Desviación:", rewards.std())

Media: -0.408
Desviación: 0.891568830206507


Es pésima la recompensa. Hay mayor cantidad de juegos perdidos a ganados, y la desviación es casi tan grande como diferencia entre empatar y ganar, es decir, el modelo no pierde y empata consistentemente (también tendríamos una media de [-0.4,-0.5]), es más aleatorio que eso.

#### **1.1.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `Blackjack`.

In [10]:
import gymnasium as gym
from stable_baselines3.common.utils import random
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

random.seed(29)

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=32000)
model.save("ppo_blackjack")

del model # remove to demonstrate saving and loading

model = PPO.load("ppo_blackjack")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.49     |
|    ep_rew_mean     | -0.55    |
| time/              |          |
|    fps             | 844      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.32        |
|    ep_rew_mean          | -0.42       |
| time/                   |             |
|    fps                  | 589         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.018417897 |
|    clip_fraction        | 0.33        |
|    clip_range           | 0.2         |
|    entropy_loss   

#### **1.1.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.1.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [11]:
epochs = 10000
rewards = []
for epoch in range(epochs):
    obs, _ = env.reset()
    done = False
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, _, _ = env.step(action)
    # Solo queremos la recompensa
    rewards.append(reward)

In [12]:
rewards = pd.Series(rewards)
print("Media:", rewards.mean())
print("Desviación:", rewards.std())

Media: -0.0623
Desviación: 0.9603702104366365


Es bastante mejor, ahora se pierde y se gana casi con la misma probabilidad, difícilmente se empatada dada la varianza tan alta. Aún así no sirve para hacerse millonario. Puede ser porque este juego esta sujeto en gran medida por el azar, aún así, el agente es capaz de equiparar las probabilidades de ganar y perder.

#### **1.1.5 Estudio de acciones (0.2 puntos)**

Genere una función que reciba un estado y retorne la accion del agente. Luego, use esta función para entregar la acción escogida frente a los siguientes escenarios:

1. Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene tiene un as
2. Suma de cartas del agente es 19, dealer muestra un 3, agente tiene tiene un as

¿Son coherentes sus acciones con las reglas del juego?

Hint: ¿A que clase de python pertenecen los estados? Pruebe a usar el método `.reset` para saberlo.

In [13]:
def explicit_player(suma:int, dealer:int, ace:bool):
    obs = np.array([suma, dealer, int(ace)])
    return "Hit" if model.predict(obs)[0] else "Stick"

tries = 10
uno = "Si la suma de cartas del agente es 6, el dealer muestra un 7, agente no tiene un As:\n"
dos = "Si la suma de cartas del agente es 19, el dealer muestra un 3, agente tiene un As:\n"
print(uno)
for _ in range(tries):
    print(explicit_player(suma=6, dealer=7, ace=False), end=' ')
print('\n\n')
print(dos)
for _ in range(tries):
    print(explicit_player(suma=19, dealer=3, ace=True), end=' ')

tries = 10000
one = []
two = []
for _ in range(tries):
    one.append(model.predict(np.array([6, 7, 0]))[0])
    two.append(model.predict(np.array([19, 3, 1]))[0])

print("\n\n")
one = pd.Series(one)
two = pd.Series(two)
print(uno)
print("Media:", one.mean())
print("Desviación:", one.std())
print('\n')
print(dos)
print("Media:", two.mean())
print("Desviación:", two.std())

Si la suma de cartas del agente es 6, el dealer muestra un 7, agente no tiene un As:

Hit Hit Hit Hit Hit Hit Hit Stick Hit Hit 


Si la suma de cartas del agente es 19, el dealer muestra un 3, agente tiene un As:

Stick Stick Stick Stick Stick Stick Stick Stick Stick Stick 


Si la suma de cartas del agente es 6, el dealer muestra un 7, agente no tiene un As:

Media: 0.7716
Desviación: 0.4198226591151442


Si la suma de cartas del agente es 19, el dealer muestra un 3, agente tiene un As:

Media: 0.0107
Desviación: 0.10289105236543011


Se concluye que las acciones son coherentes con las reglas del juego. Estado muy cerca del 21 resulta más obvio querer quedarse (Stick) en el número. Lo que se muestra en la media y la desviación del agente para este escenario, rondando siempre esta acción.

Para el primer caso resulta más extraña la jugada y esta mucho más sujeta a la suerte que tenga el dealer, es decir, dada la mala mano del agente, este decide apostar por que el repartidor supere el 21, dandole la victoria. Esto se refleja en la media encontrada y su desviación. A pesar de esto existe una desviación estandar superior al otro escenario, esto porque se entiende que el numero 6 sigue siendo bajo y las probabilidades de superar el 21 con la siguiente carta son nulas. 

Los estados son clase numpy.array

### **1.2 LunarLander**

<p align="center">
  <img src="https://i.redd.it/097t6tk29zf51.jpg"
" width="400">
</p>

Similar a la sección 2.1, en esta sección usted se encargará de implementar una gente de RL que pueda resolver el ambiente `LunarLander`.

Comencemos preparando el ambiente:


In [14]:
import gymnasium as gym
env = gym.make("LunarLander-v2", render_mode="rgb_array", continuous=False, max_episode_steps=200) # notar el parámetro continuous = True}
# Tuve un error con continuous=True. El método de step() no funcionaba bien. Cambiando a discreto lo arreglo.

Noten que se especifica el parámetro `continuous = True`. ¿Que implicancias tiene esto sobre el ambiente?

Además, se le facilita la función `export_gif` para el ejercicio 2.2.4:

In [15]:
import imageio
import numpy as np

def export_gif(model, n = 5):
  '''
  función que exporta a gif el comportamiento del agente en n episodios
  '''
  images = []
  for episode in range(n):
    obs = model.env.reset()
    img = model.env.render()
    done = False
    while not done:
      images.append(img)
      action, _ = model.predict(obs)
      obs, reward, done, info = model.env.step(action)
      img = model.env.render(mode="rgb_array")

  imageio.mimsave("agent_performance.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=29)

#### **1.2.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [LunarLander](https://gymnasium.farama.org/environments/box2d/lunar_lander/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas. ¿Como se distinguen las acciones de este ambiente en comparación a `Blackjack`?

Nota: recuerde que se especificó el parámetro `continuous = True`

Los estados son continuos y por lo tanto son teoricamente infinitos.

Las observaciones entregadas se componen de una tupla de ocho valores, 6 continuos para nuestro experimento, y dos boolean

* 0: Posición eje X (float)
* 1: Posición eje Y (float)
* 2: Velocidad eje X (float)
* 3: Velocidad eje Y (float)
* 4: Angulo de la nave (float)
* 5: Velocidad Angular (float)
* 6: Pata 1 en contacto con las plataforma de aterrizaje (int)
* 7: Pata 2 en contacto con las plataforma de aterrizaje (int)

La acción es un valor 0 al 3 (int)
* 0: No acción
* 1: Prender motor izquierdo
* 2: Prender motor central
* 3: Prender motor derecho

Las recompensas (int)

* Aumenta/decrece de acuerdo a la cercania/lejania de la nave a la plataforma de aterrizaje.
* Aumenta/decrece según mayor/menor sea su velocidad.
* Decrece si la nave se ladea (a mayor angulo).
* 10: por cada pata tocando la plataforma.
* 0.03: por cada cuadro donde alguno de los motores laterales estan encendidos.
* 0.3: por cada cuadro donde el motor central esta encendido.
* +100/-100 por aterrizar exitosamente/estrellarse.

Juego terminado

* La nave se estrella.
* La plataforma de aterrizaje sale del cuadro (X>1).
* La nave no está "despierta". Según los documentos de Box2D, un cuerpo que no está despierto es un cuerpo que no se mueve y no choca con ningún otro cuerpo.

#### **1.2.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 10 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política?

In [16]:
env.reset()

(array([ 0.00321198,  1.4134482 ,  0.3253267 ,  0.1123637 , -0.00371512,
        -0.07369135,  0.        ,  0.        ], dtype=float32),
 {})

In [17]:
env.action_space.sample()

np.int64(1)

In [18]:
epochs = 20
rewards = []
for epoch in range(epochs):
    obs, _ = env.reset()
    done = False
    reward = 0
    while not done:
        # action, _ = model.predict(obs)
        action = np.random.randint(0,4)
        obs, r, done, _, _ = env.step(env.action_space.sample())
        reward += r
    # Solo queremos la recompensa
    rewards.append(reward)

In [19]:
rewards = pd.Series(rewards)
print("Media:", rewards.mean())
print("Desviación:", rewards.std())

Media: -157.72839748928004
Desviación: 96.43861160222237


Mala. Que la media sea casi el doble que el puntaje obtenido al estrellarse, es de suponer que la gran mayoría de los episodios la nave se estrello

#### **1.2.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `LunarLander` **usando 10000 timesteps de entrenamiento**.

In [20]:
def train_ppo(timestepts=10000, lr=0.0003, batch_size=64, save=False):
    model = PPO("MlpPolicy", env, verbose=1, learning_rate=lr, batch_size=batch_size)
    model.learn(total_timesteps=timestepts, )
    if save:
        model.save("ppo_lunarlander")
    return model
train_ppo()
model = PPO.load("ppo_lunarlander")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 88.1     |
|    ep_rew_mean     | -145     |
| time/              |          |
|    fps             | 976      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 95.2         |
|    ep_rew_mean          | -197         |
| time/                   |              |
|    fps                  | 707          |
|    iterations           | 2            |
|    time_elapsed         | 5            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0077093877 |
|    clip_fraction        | 0.0486       |
|    clip_range           | 0.2          |
|    en

#### **1.2.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.2.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [ ]:
epochs = 10
rewards = []
for epoch in range(epochs):
    obs, _ = env.reset()
    done = False
    reward = 0
    while not done:
        # action, _ = model.predict(obs)
        action, _ = model.predict(obs)
        obs, r, done, _, _ = env.step(action)
        reward += r
    # Solo queremos la recompensa
    rewards.append(reward)

In [ ]:
rewards = pd.Series(rewards)
print("Media:", rewards.mean())
print("Desviación:", rewards.std())

Es mejor, pero no mucho mejor. La media es mayor pero igual cercana a la recompensa por estrellar (-100). La desviación es menor, se puede asumir que hubieron algunos episodios donde la nave pudo mantenerse y/o acercarse a la plataforma de aterrizaje antes de estrellar.

#### **1.2.5 Optimización de modelo (0.2 puntos)**

Repita los ejercicios 1.2.3 y 1.2.4 hasta obtener un nivel de recompensas promedio mayor a 50. Para esto, puede cambiar manualmente parámetros como:
- `total_timesteps`
- `learning_rate`
- `batch_size`

Una vez optimizado el modelo, use la función `export_gif` para estudiar el comportamiento de su agente en la resolución del ambiente y comente sobre sus resultados.

Adjunte el gif generado en su entrega (mejor aún si además adjuntan el gif en el markdown).

In [ ]:
model = train_ppo(timestepts=100000, lr=0.0001, batch_size=128, save=True)

In [ ]:
epochs = 10
rewards = []
for epoch in range(epochs):
    obs, _ = env.reset()
    done = False
    reward = 0
    while not done:
        # action, _ = model.predict(obs)
        action, _ = model.predict(obs)
        obs, r, done, _, _ = env.step(action)
        reward += r
    # Solo queremos la recompensa
    rewards.append(reward)

In [ ]:
rewards = pd.Series(rewards)
print("Media:", rewards.mean())
print("Desviación:", rewards.std())

In [ ]:
export_gif(model)

<p align="center">
  <img src="./lab11/"
" width="400">
</p>

## **2. Large Language Models (4.0 puntos)**

En esta sección se enfocarán en habilitar un Chatbot que nos permita responder preguntas útiles a través de LLMs.

### **2.0 Configuración Inicial**

<p align="center">
  <img src="https://media1.tenor.com/m/uqAs9atZH58AAAAd/config-config-issue.gif"
" width="400">
</p>

Como siempre, cargamos todas nuestras API KEY al entorno:

In [ ]:
# import getpass
# import os

# if "GOOGLE_API_KEY" not in os.environ:
#     os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

# if "TAVILY_API_KEY" not in os.environ:
#     os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

### **2.1 Retrieval Augmented Generation (1.5 puntos)**

<p align="center">
  <img src="https://y.yarn.co/218aaa02-c47e-4ec9-b1c9-07792a06a88f_text.gif"
" width="400">
</p>

El objetivo de esta subsección es que habiliten un chatbot que pueda responder preguntas usando información contenida en documentos PDF a través de **Retrieval Augmented Generation.**

#### **2.1.1 Reunir Documentos (0 puntos)**

Reuna documentos PDF sobre los que hacer preguntas siguiendo las siguientes instrucciones:
  - 2 documentos .pdf como mínimo.
  - 50 páginas de contenido como mínimo entre todos los documentos.
  - Ideas para documentos: Documentos relacionados a temas académicos, laborales o de ocio. Aprovechen este ejercicio para construir algo útil y/o relevante para ustedes!
  - Deben ocupar documentos reales, no pueden utilizar los mismos de la clase.
  - Deben registrar sus documentos en la siguiente [planilla](https://docs.google.com/spreadsheets/d/1Hy1w_dOiG2UCHJ8muyxhdKPZEPrrL7BNHm6E90imIIM/edit?usp=sharing). **NO PUEDEN USAR LOS MISMOS DOCUMENTOS QUE OTRO GRUPO**
  - **Recuerden adjuntar los documentos en su entrega**.

In [ ]:
%pip install --upgrade --quiet PyPDF2

In [ ]:
import PyPDF2

doc_paths = [] # rellenar con los path a sus documentos

assert len(doc_paths) >= 2, "Deben adjuntar un mínimo de 2 documentos"

total_paginas = sum(len(PyPDF2.PdfReader(open(doc, "rb")).pages) for doc in doc_paths)
assert total_paginas >= 50, f"Páginas insuficientes: {total_paginas}"

#### **2.1.2 Vectorizar Documentos (0.2 puntos)**

Vectorice los documentos y almacene sus representaciones de manera acorde.

#### **2.1.3 Habilitar RAG (0.3 puntos)**

Habilite la solución RAG a través de una *chain* y guárdela en una variable.

#### **2.1.4 Verificación de respuestas (0.5 puntos)**

Genere un listado de 3 tuplas ("pregunta", "respuesta correcta") y analice la respuesta de su solución para cada una. ¿Su solución RAG entrega las respuestas que esperaba?

Ejemplo de tupla:
- Pregunta: ¿Quién es el presidente de Chile?
- Respuesta correcta: El presidente de Chile es Gabriel Boric

#### **2.1.5 Sensibilidad de Hiperparámetros (0.5 puntos)**

Extienda el análisis del punto 2.1.4 analizando cómo cambian las respuestas entregadas cambiando los siguientes hiperparámetros:
- `Tamaño del chunk`. (*¿Cómo repercute que los chunks sean mas grandes o chicos?*)
- `La cantidad de chunks recuperados`. (*¿Qué pasa si se devuelven muchos/pocos chunks?*)
- `El tipo de búsqueda`. (*¿Cómo afecta el tipo de búsqueda a las respuestas de mi RAG?*)

### **2.2 Agentes (1.0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/rcqnN2aJCSEAAAAd/secret-agent-man.gif"
" width="400">
</p>

Similar a la sección anterior, en esta sección se busca habilitar **Agentes** para obtener información a través de tools y así responder la pregunta del usuario.

#### **2.2.1 Tool de Tavily (0.2 puntos)**

Generar una *tool* que pueda hacer consultas al motor de búsqueda **Tavily**.

#### **2.2.2 Tool de Wikipedia (0.2 puntos)**

Generar una *tool* que pueda hacer consultas a **Wikipedia**.

*Hint: Le puede ser de ayuda el siguiente [link](https://python.langchain.com/v0.1/docs/modules/tools/).*

#### **2.2.3 Crear Agente (0.3 puntos)**

Crear un agente que pueda responder preguntas preguntas usando las *tools* antes generadas. Asegúrese que su agente responda en español. Por último, guarde el agente en una variable.

#### **2.2.4 Verificación de respuestas (0.3 puntos)**

Pruebe el funcionamiento de su agente y asegúrese que el agente esté ocupando correctamente las tools disponibles. ¿En qué casos el agente debería ocupar la tool de Tavily? ¿En qué casos debería ocupar la tool de Wikipedia?

### **2.3 Multi Agente (1.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/r7QMJLxU4BoAAAAd/this-is-getting-out-of-hand-star-wars.gif"
" width="450">
</p>

El objetivo de esta subsección es encapsular las funcionalidades creadas en una solución multiagente con un **supervisor**.


#### **2.3.1 Generando Tools (0.5 puntos)**

Transforme la solución RAG de la sección 2.1 y el agente de la sección 2.2 a *tools* (una tool por cada uno).

#### **2.3.2 Agente Supervisor (0.5 puntos)**

Habilite un agente que tenga acceso a las tools del punto anterior y pueda responder preguntas relacionadas. Almacene este agente en una variable llamada supervisor.

#### **2.3.3 Verificación de respuestas (0.25 puntos)**

Pruebe el funcionamiento de su agente repitiendo las preguntas realizadas en las secciones 2.1.4 y 2.2.4 y comente sus resultados. ¿Cómo varían las respuestas bajo este enfoque?

#### **2.3.4 Análisis (0.25 puntos)**

¿Qué diferencias tiene este enfoque con la solución *Router* vista en clases? Nombre al menos una ventaja y desventaja.

`escriba su respuesta acá`

### **2.4 Memoria (Bonus +0.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/Gs95aiElrscAAAAd/memory-unlocked-ratatouille-critic.gif"
" width="400">
</p>

Una de las principales falencias de las soluciones que hemos visto hasta ahora es que nuestro chat no responde las interacciones anteriores, por ejemplo:

- Pregunta 1: "Hola! mi nombre es Sebastián"
  - Respuesta esperada: "Hola Sebastián! ..."
- Pregunta 2: "Cual es mi nombre?"
  - Respuesta actual: "Lo siento pero no conozco tu nombre :("
  - **Respuesta esperada: "Tu nombre es Sebastián"**

Para solucionar esto, se les solicita agregar un componente de **memoria** a la solución entregada en el punto 2.3.

**Nota: El Bonus es válido <u>sólo para la sección 2 de Large Language Models.</u>**

### **2.5 Despliegue (0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/IytHqOp52EsAAAAd/you-get-a-deploy-deploy.gif"
" width="400">
</p>

Una vez tengan los puntos anteriores finalizados, toca la etapa de dar a conocer lo que hicimos! Para eso, vamos a desplegar nuestro modelo a través de `gradio`, una librería especializada en el levantamiento rápido de demos basadas en ML.

Primero instalamos la librería:

In [ ]:
%pip install --upgrade --quiet gradio

Luego sólo deben ejecutar el siguiente código e interactuar con la interfaz a través del notebook o del link generado:

In [ ]:
import gradio as gr
import time

def agent_response(message, history):
  '''
  Función para gradio, recibe mensaje e historial, devuelte la respuesta del chatbot.
  '''
  # get chatbot response
  response = ... # rellenar con la respuesta de su chat

  # assert
  assert type(response) == str, "output de route_question debe ser string"

  # "streaming" response
  for i in range(len(response)):
    time.sleep(0.015)
    yield response[: i+1]

gr.ChatInterface(
    agent_response,
    type="messages",
    title="Chatbot MDS7202", # Pueden cambiar esto si lo desean
    description="Hola! Soy un chatbot muy útil :)", # también la descripción
    theme="soft",
    ).launch(
        share=True, # pueden compartir el link a sus amig@s para que interactuen con su chat!
        debug = False,
        )